# Phénomène de Runge

## Formulation de Newton

Introduisons les différences divisées par la récurrence suivante : 

$$
\begin{array}{rcl}
y[x_i] &:=& y_i \vphantom{\left(\frac{\delta y[x_j,x_k] -\delta y[x_i,x_j]}{x_k-x_i}\right)^2} \\
\delta y[x_i,x_j] &:= & \displaystyle \frac{y[x_j] -y[x_i]}{x_j-x_i} \vphantom{\left(\frac{\delta y[x_j,x_k] -\delta y[x_i,x_j]}{x_k-x_i}\right)^2} \\
\delta^2 y[x_i,x_j,x_k] &:= &\displaystyle \vphantom{\left(\frac{\delta y[x_j,x_k] -\delta y[x_i,x_j]}{x_k-x_i}\right)^2} \frac{\delta y[x_j,x_k] -\delta y[x_i,x_j]}{x_k-x_i}\\
\vdots&&\vdots\\
\delta^n y[x_{i_0},x_{i_1},\ldots,x_{i_n}] &:= & \displaystyle \frac{\delta^{n-1} y[x_{i_1},x_{i_1},\ldots,x_{i_n}] -\delta^{n-1} y[x_{i_0},x_{i_1},\ldots,x_{i_{n-1}}]}{x_{i_n}-x_{i_0}}. \vphantom{\left(\frac{\delta y[x_j,x_k] -\delta y[x_i,x_j]}{x_k-x_i}\right)^2} \\
\end{array}
$$


Le polynôme d’interpolation de degré $n$ qui passe par les $n+1$ points $(x_0,y_0),(x_1,y_1), \ldots (x_n,y_n)$, où les $x_i$ sont distincts, est l'unique polynôme donné par l’expression :

$$
\begin{aligned}
p_n(x) =y[x_0]  {} & + (x-x_0)\delta y[x_0,x_1]  + (x-x_0)(x-x_1)\delta^2 y[x_0,x_1,x_2]+ \ldots \\
             & + (x-x_0)\ldots(x-x_{n-1}) \delta^{n} y[x_{0},x_{i},\ldots,x_{n}].
\end{aligned}
$$

In [ ]:
import numpy as np

def compute_divided_diff_coef(x, y):
    n = x.size
    coef = y.copy()
    for i in range(1,n):
        coef[i:] = (coef[i:] - coef[i-1:-1])/(x[i:] - x[:-i])
    return coef

def poly_newton_interp(coef, xi, x):
    n = coef.size
    p = np.zeros(x.size)
    for i in range(n-1,0,-1):
        p = (coef[i]+p) * (x-xi[i-1])
    p = p +  coef[0] 
    return p

def newton_interp(xi, yi):
    coef = compute_divided_diff_coef(xi, yi)
    return lambda x: poly_newton_interp(coef, xi, x)

## Exemple

We consider the function $f(x) = \displaystyle \frac{1}{(1 + 25x^2)}$.

In [ ]:
def f(x):
    return 1/(1+25*x*x)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

xmin = -1.0
xmax = 1.0

# maximal degree of Newton interpolating polynomial
n_max = 30

# array of degree
arr_n = np.arange(1, n_max)

# compute for each degree xi, yi and pn
xi_equi = []
yi_equi = []
p_equi = []
for i, ni in enumerate(arr_n):
    xi_equi.append(np.linspace(xmin, xmax, ni+1))
    yi_equi.append(f(xi_equi[i]))
    p_equi.append(newton_interp(xi_equi[i], yi_equi[i]))
    
# Create figure
fig = go.Figure()

# x use to display f and pn
xplot = np.linspace(xmin, xmax, 500)

# add f(x) plot
fig.add_trace(go.Scatter(visible=True, x=xplot, y=f(xplot), name="f(x)"))

# add yi and pn(x) invisible plots
for i, ni in enumerate(arr_n):
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=p_equi[i](xplot), name=f"p{ni-1}(x)", marker=dict(color='forestgreen')))
    fig.add_trace(go.Scatter(visible=False, x=xi_equi[i], y=yi_equi[i], mode='markers', name="interpolating points", marker=dict(color='forestgreen')))

# Make plot visible for n=1
fig.data[1].visible = True
fig.data[2].visible = True

# Create and add slider
steps = []
for i, ni in enumerate(arr_n):
    step = dict(method="update", label = f" {ni+1}",
                args=[{"visible": [el==0 or el==2*i+1 or el==2*i+2 for el in range(len(fig.data))]}])
    steps.append(step)
        
sliders = [dict(
    currentvalue={"prefix": "nb points: "},
    steps=steps
    )]

fig.update_layout(sliders=sliders)
fig.update_yaxes(range=[-.25, 1.25])

fig.show()

Lorsque le nombre de points d'interpolation augmentent, on remarqe que le polynôme se met à osciller avec une amplitude de plus en plus grande. C'est le **phénomène de Runge**.

## Point de Tchebychef

In [ ]:
def cheb_points(xmin, xmax, n):
    x = np.zeros(n+1)
    for i in range(n+1):
        x[i] = (xmax+xmin)/2 + ((xmax-xmin)/2) * np.cos(((2*i+1)*np.pi)/(2*n + 2))
    return x

In [ ]:
# compute for each degree xi, yi and pn
xi_cheb = []
yi_cheb = []
p_cheb = []
for i, ni in enumerate(arr_n):
    xi_cheb.append(cheb_points(xmin, xmax, ni))
    yi_cheb.append(f(xi_cheb[i]))
    p_cheb.append(newton_interp(xi_cheb[i], yi_cheb[i]))

fig = make_subplots(rows=1, cols=2, subplot_titles=("Equidistants points", "Chebychev points"))

# add f(x) plot
fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name="f(x)", legendgroup="f", marker=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name="f(x)", legendgroup="f", showlegend=False, marker=dict(color='blue')), row=1, col=2)


# add yi and pn(x) invisible plots
for i, ni in enumerate(arr_n):
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=p_equi[i](xplot), name=f"p{ni-1}(x)", 
                            marker=dict(color='forestgreen')), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=xi_equi[i], y=yi_equi[i], mode='markers', showlegend=False,
                             marker=dict(color='forestgreen')), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=p_cheb[i](xplot), showlegend=False,
                              marker=dict(color='forestgreen')), row=1, col=2)
    fig.add_trace(go.Scatter(visible=False, x=xi_cheb[i], y=yi_cheb[i], mode='markers', showlegend=False,
                             marker=dict(color='forestgreen')), row=1, col=2)
    
# Make plot visible for n=1
fig.data[2].visible = True
fig.data[3].visible = True
fig.data[4].visible = True
fig.data[5].visible = True

# Create and add slider
steps = []
for i, ni in enumerate(arr_n):
    step = dict(method="update", label = f" {ni+1}",
                args=[{"visible": [el==0 or el==1 or el==4*i+2 or el==4*i+3 or el==4*i+4 or el==4*i+5 for el in range(len(fig.data))]}])
    steps.append(step)
        
sliders = [dict(
    currentvalue={"prefix": "nb points: "},
    steps=steps
    )]

fig.update_layout(sliders=sliders)
fig.update_yaxes(range=[-.25, 1.25])
fig.show()